# 🚀 Quick Start: RAG in 5 Minutes

**Already have Ollama installed with the models?** Jump straight to the "Load Dataset" section and run cells sequentially.

**First time?** Complete setup below (3 terminal commands), then run all cells.

```bash
# 1. Pull embedding model (creates vector representations)
ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf

# 2. Pull language model (generates answers)
ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF

# 3. Install Python packages
pip install ollama datasets jupyter
```

Then open this notebook and run all cells from top to bottom. No database setup needed!

---

# Foundation 01: Basic RAG (In-Memory)

This notebook demonstrates a simple Retrieval-Augmented Generation (RAG) system using Simple Wikipedia articles with **in-memory storage** and optional JSON file caching.

This is the **simple version** with minimal dependencies - perfect for learning RAG fundamentals without database setup.

**Ready for persistent storage?** See `foundation/02-rag-postgresql-persistent.ipynb` for the PostgreSQL version with durable embeddings and registry integration.

## Setup and Installation

Before running this notebook, you need to:

1. Install Ollama from [ollama.com](https://ollama.com/)
2. Download the required models by running these commands in your terminal:

```bash
ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF
```

3. Install the required Python packages:

```bash
pip install ollama datasets jupyter
```

## Learning Progression

- ✅ **You are here:** foundation/01 - In-memory RAG basics
- ⏭️  **Next:** foundation/02 - PostgreSQL persistent storage with registry
- 🎯 **Path:** See `LEARNING_ROADMAP.md` for complete learning paths

## Import Dependencies

In [ ]:
import ollama
from datasets import load_dataset
import json
import sys
import math

## Configuration

Set the target dataset size. The script will download articles until it reaches approximately this size.

In [ ]:
# Target dataset size in MB (adjust as needed: 10, 20, 30, 40, 50)
TARGET_SIZE_MB = 10

# Maximum chunk size in characters (for splitting long articles)
MAX_CHUNK_SIZE = 1000

# Whether to save the dataset locally for reuse
SAVE_LOCALLY = True
LOCAL_DATASET_PATH = f'wikipedia_dataset_{TARGET_SIZE_MB}mb.json'

## Load and Filter the Wikipedia Dataset

We'll use Simple Wikipedia, which has cleaner, more concise articles. The dataset will be filtered to approximately your target size.

In [ ]:
def estimate_size_mb(text):
    """Estimate the size of text in megabytes."""
    return sys.getsizeof(text) / (1024 * 1024)

def chunk_text(text, max_size=1000):
    """Split text into chunks of approximately max_size characters.
    
    Why chunking?
    - Long documents don't fit in embedding models efficiently
    - Smaller chunks retrieve more precise context
    - Overlapping chunks preserve semantic continuity
    
    Tries to break at paragraph boundaries when possible.
    """
    if len(text) <= max_size:
        return [text]
    
    chunks = []
    paragraphs = text.split('\n\n')
    current_chunk = ''
    
    for paragraph in paragraphs:
        # If adding this paragraph would exceed max_size
        if len(current_chunk) + len(paragraph) > max_size:
            if current_chunk:  # Save current chunk if not empty
                chunks.append(current_chunk.strip())
                current_chunk = ''
            
            # If single paragraph is too large, split it
            if len(paragraph) > max_size:
                sentences = paragraph.split('. ')
                for sentence in sentences:
                    if len(current_chunk) + len(sentence) > max_size:
                        if current_chunk:
                            chunks.append(current_chunk.strip())
                        current_chunk = sentence + '. '
                    else:
                        current_chunk += sentence + '. '
            else:
                current_chunk = paragraph
        else:
            current_chunk += '\n\n' + paragraph if current_chunk else paragraph
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def load_wikipedia_dataset(target_size_mb, local_path=None):
    """Load and filter Wikipedia dataset to target size.
    
    This is Step 1 of the RAG Pipeline:
    📚 DATASET LOADING: Prepare the knowledge source
    
    The RAG pipeline has 3 main phases:
    1. 📚 Indexing Phase (this function): Load and chunk documents
    2. 🔍 Retrieval Phase (retrieve function): Find relevant chunks for query
    3. 💬 Generation Phase (ask_question function): Generate answer using context
    
    Args:
        target_size_mb: Target dataset size in megabytes
        local_path: Path to save/load dataset locally
    
    Returns:
        List of text chunks ready for embedding
    """
    # Try to load from local cache first (faster than re-downloading)
    if local_path:
        try:
            print(f'Attempting to load cached dataset from {local_path}...')
            with open(local_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f'✓ Loaded {len(data["chunks"])} chunks from cache')
                print(f'  Estimated size: {data["size_mb"]:.2f} MB')
                return data['chunks']
        except FileNotFoundError:
            print('No cached dataset found, downloading from HuggingFace...')
    
    # Load Simple Wikipedia dataset (cleaner and more concise than full Wikipedia)
    print('Loading Simple Wikipedia dataset (this may take a minute)...')
    dataset = load_dataset('wikimedia/wikipedia', '20231101.simple', split='train', streaming=True)
    
    chunks = []
    current_size_mb = 0
    target_bytes = target_size_mb * 1024 * 1024
    article_count = 0
    
    print(f'\nCollecting articles (target: {target_size_mb} MB)...')
    
    for article in dataset:
        # Skip very short articles (not enough context)
        if len(article['text']) < 200:
            continue
        
        # Create metadata-enriched chunks (add title for context)
        article_chunks = chunk_text(article['text'], MAX_CHUNK_SIZE)
        
        for chunk in article_chunks:
            # Add title context to help with retrieval
            # Example: "Article: France\n\nParis is the capital of France..."
            enriched_chunk = f"Article: {article['title']}\n\n{chunk}"
            chunk_size = sys.getsizeof(enriched_chunk)
            
            chunks.append(enriched_chunk)
            current_size_mb += chunk_size
            
            # Check if we've reached target size
            if current_size_mb >= target_bytes:
                break
        
        article_count += 1
        
        # Progress update every 50 articles
        if article_count % 50 == 0:
            print(f'  Progress: {current_size_mb / (1024*1024):.2f} MB ({article_count} articles, {len(chunks)} chunks)')
        
        if current_size_mb >= target_bytes:
            break
    
    final_size_mb = current_size_mb / (1024 * 1024)
    print(f'\n✓ Dataset loaded: {len(chunks)} chunks from {article_count} articles')
    print(f'  Estimated size: {final_size_mb:.2f} MB')
    
    # Save locally if requested (avoids re-downloading on notebook restart)
    if local_path:
        print(f'\nSaving dataset to {local_path}...')
        with open(local_path, 'w', encoding='utf-8') as f:
            json.dump({
                'size_mb': final_size_mb,
                'chunk_count': len(chunks),
                'article_count': article_count,
                'chunks': chunks
            }, f, ensure_ascii=False)
        print('✓ Dataset saved for future use')
    
    return chunks

# Load the dataset
dataset = load_wikipedia_dataset(
    TARGET_SIZE_MB, 
    LOCAL_DATASET_PATH if SAVE_LOCALLY else None
)

print(f'\nReady to build vector database with {len(dataset)} chunks!')

## Sample Data

Let's look at a few examples from our dataset:

In [ ]:
print('Sample chunks from the dataset:\n')
for i, chunk in enumerate(dataset[:3]):
    print(f'--- Chunk {i+1} ---')
    print(chunk[:300] + '...' if len(chunk) > 300 else chunk)
    print()

## Configure Models

We'll use two models:
- **Embedding Model**: Converts text into vector representations
- **Language Model**: Generates responses based on retrieved context

In [ ]:
EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

## Implement the Vector Database

### Indexing Phase

In the indexing phase, we:
1. Break the dataset into chunks (already done during loading)
2. Calculate embedding vectors for each chunk
3. Store chunks with their embeddings in our vector database

Each element in `VECTOR_DB` will be a tuple: `(chunk, embedding)`

The embedding is a list of floats, for example: `[0.1, 0.04, -0.34, 0.21, ...]`

**Note**: This may take a few minutes depending on your dataset size.

In [ ]:
# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# embedding = [0.12, -0.45, 0.78, ...] (768 dimensions for our model)
VECTOR_DB = []

def add_chunk_to_database(chunk):
    """Add a chunk and its embedding to the vector database.
    
    This is the critical step that makes RAG work:
    
    TEXT CHUNK              EMBEDDING MODEL            VECTOR (768 numbers)
    "Paris is the      →    (BGE Model)        →    [0.12, -0.45, 0.78, ...]
     capital of France"     
    
    The embedding captures semantic meaning. Similar chunks get similar vectors!
    """
    # Generate embedding vector from text (768-dimensional for BGE model)
    embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
    # Store both the original text and its vector representation
    VECTOR_DB.append((chunk, embedding))

Now let's populate our vector database with all chunks from the dataset:

In [ ]:
print(f'Building vector database with {len(dataset)} chunks...')
print('This may take a few minutes...\n')

for i, chunk in enumerate(dataset):
    add_chunk_to_database(chunk)
    
    # Progress update every 50 chunks
    if (i + 1) % 50 == 0:
        print(f'Embedded {i+1}/{len(dataset)} chunks ({(i+1)/len(dataset)*100:.1f}%)')

print(f'\n✓ Vector database ready with {len(VECTOR_DB)} embeddings!')

## Implement the Retrieval Function

### Cosine Similarity

To find the most relevant chunks, we need to compare vector similarity. We'll use cosine similarity, which measures how "close" two vectors are in the vector space. Higher cosine similarity means more similar meaning.

In [ ]:
def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors."""
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x ** 2 for x in a]) ** 0.5
    norm_b = sum([x ** 2 for x in b]) ** 0.5
    return dot_product / (norm_a * norm_b)

### Retrieval Function

The retrieval function:
1. Converts the query into an embedding vector
2. Compares it against all vectors in the database
3. Returns the top N most relevant chunks

In [ ]:
def retrieve(query, top_n=3):
    """Retrieve the top N most relevant chunks for a given query.
    
    This is the RETRIEVAL phase of RAG:
    
    1. Convert query to embedding  →  [query_vector]
    2. Compare to all stored embeddings using cosine similarity
    3. Return top N most similar chunks
    
    Why it works: Similar meanings produce similar vectors!
    If you ask "What is the capital of France?"
    It will find chunks about Paris because they're semantically similar.
    """
    # Step 1: Convert the user's question into an embedding vector
    query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
    
    # Step 2: Calculate similarity between query and all chunks
    similarities = []
    for chunk, embedding in VECTOR_DB:
        # Cosine similarity measures angle between vectors (0 = orthogonal, 1 = identical)
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk, similarity))
    
    # Step 3: Sort by similarity (descending) and return top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

## Generation Phase

In the generation phase, the chatbot generates a response based on the retrieved knowledge. We construct a prompt that includes the relevant chunks and instruct the model to only use that context.

In [ ]:
def ask_question(query, top_n=3, verbose=True):
    """Ask a question and get a response based on retrieved knowledge.
    
    This is the GENERATION phase of RAG:
    
    USER QUESTION
         ↓
    [Retrieve relevant chunks]  ← uses semantic search
         ↓
    [Build context prompt]  ← "Use only this knowledge to answer"
         ↓
    [Generate response with LLM]  ← Llama model reads context and answers
         ↓
    ANSWER (grounded in Wikipedia)
    
    Args:
        query: The question to ask
        top_n: Number of relevant chunks to retrieve
        verbose: Whether to print retrieved knowledge and response
    
    Returns:
        The chatbot's response as a string
    """
    # Step 1: Retrieve relevant knowledge
    retrieved_knowledge = retrieve(query, top_n=top_n)
    
    if verbose:
        print('Retrieved knowledge:')
        for i, (chunk, similarity) in enumerate(retrieved_knowledge):
            # Extract title from chunk
            title_line = chunk.split('\n')[0]
            preview = chunk[:200].replace('\n', ' ') + '...' if len(chunk) > 200 else chunk
            # Similarity score: 0 = no match, 1 = perfect match
            print(f'  [{i+1}] (similarity: {similarity:.3f}) {preview}')
        print()
    
    # Step 2: Build instruction prompt with retrieved context
    # This tells the model: "Here is knowledge. Answer the question using ONLY this."
    instruction_prompt = f'''You are a helpful chatbot that answers questions based on Wikipedia articles.
Use only the following pieces of context to answer the question. Don't make up any new information.
If the context doesn't contain enough information to answer the question, say so.

Context:
{chr(10).join([f'{i+1}. {chunk.strip()}' for i, (chunk, _) in enumerate(retrieved_knowledge)])}
'''
    
    # Step 3: Send to language model for generation
    # The LLM reads the retrieved context and generates a grounded answer
    stream = ollama.chat(
        model=LANGUAGE_MODEL,
        messages=[
            {'role': 'system', 'content': instruction_prompt},
            {'role': 'user', 'content': query},
        ],
        stream=True,  # Stream response token-by-token for faster feedback
    )
    
    # Step 4: Collect and display the response
    if verbose:
        print('Chatbot response:')
    
    response = ''
    for chunk in stream:
        content = chunk['message']['content']
        response += content
        if verbose:
            print(content, end='', flush=True)  # Print as it arrives
    
    if verbose:
        print('\n')  # ensure a newline after the streamed response
    
    return response

## Try It Out!

Now let's ask some questions. The quality of answers will depend on which articles were included in your dataset sample.

In [ ]:
ask_question("What is the capital of France?")

In [ ]:
ask_question("Tell me about Albert Einstein")

In [ ]:
ask_question("What is Python programming language?")

In [ ]:
ask_question("How does photosynthesis work?")

## Interactive Chat

You can also use this cell to ask your own questions:

In [ ]:
# Ask your own question here
your_question = "What is the solar system?"
ask_question(your_question)

## Export Embeddings for Vector Databases

Export your embeddings to use with various vector database platforms. Choose the format that matches your target platform:

In [ ]:
def export_embeddings(chunks=None, embeddings=None, output_path='embeddings_export.json', format='generic'):
    """
    Export embeddings in a generic format compatible with multiple vector databases.
    
    This function removes vendor lock-in by providing a standard export format
    that works with any PostgreSQL-compatible vector database.
    
    Args:
        chunks: List of text chunks (optional, will use VECTOR_DB if not provided)
        embeddings: List of embedding vectors (optional, will use VECTOR_DB if not provided)
        output_path (str): Path to output JSON or SQL file
        format (str): Export format - 'generic' (default), 'pgvector', or 'pinecone'
    
    Returns:
        dict: Export statistics (count, dimension, file_size_mb, format, path)
    
    Supports:
        - PostgreSQL with pgvector (local, Neon, Supabase, RDS)
        - Pinecone vector database
        - Generic JSON for custom integrations
        
    Examples:
        # Export as generic JSON
        stats = export_embeddings(format='generic')
        
        # Export as PostgreSQL INSERT statements
        stats = export_embeddings(format='pgvector')
        
        # Export for Pinecone
        stats = export_embeddings(format='pinecone')
    """
    # Use provided chunks/embeddings or default to VECTOR_DB
    if chunks is None or embeddings is None:
        if not VECTOR_DB:
            raise ValueError('No embeddings available. Generate embeddings first or provide chunks and embeddings.')
        chunks = [chunk for chunk, _ in VECTOR_DB]
        embeddings = [emb for _, emb in VECTOR_DB]
    
    if len(chunks) != len(embeddings):
        raise ValueError('chunks and embeddings must have the same length')
    
    embedding_dimension = len(embeddings[0]) if embeddings else 0
    
    if format == 'generic':
        # Generic JSON format: standard structure without vendor lock-in
        export_data = {
            'metadata': {
                'model': EMBEDDING_MODEL,
                'dimension': embedding_dimension,
                'count': len(embeddings),
                'created_at': __import__('datetime').datetime.now().isoformat() + 'Z',
                'format_type': 'generic'
            },
            'embeddings': [
                {
                    'id': f'chunk_{i}',
                    'vector': embedding,
                    'metadata': {
                        'text': chunk,
                        'source': chunk.split('\n')[0].replace('Article: ', '') if chunk.startswith('Article: ') else 'unknown'
                    }
                }
                for i, (chunk, embedding) in enumerate(zip(chunks, embeddings))
            ]
        }
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)
        
        file_size_mb = __import__('os').path.getsize(output_path) / (1024 * 1024)
        
        print(f'✓ Exported {len(embeddings)} embeddings in generic JSON format')
        print(f'  Output: {output_path}')
        print(f'  File size: {file_size_mb:.2f} MB')
        print(f'  Dimension: {embedding_dimension}')
        print(f'\nThis format works with:')
        print(f'  - PostgreSQL with pgvector (via JSON import)')
        print(f'  - Neon PostgreSQL')
        print(f'  - Supabase (PostgreSQL + pgvector)')
        print(f'  - AWS RDS with pgvector')
        print(f'  - Custom vector database integrations')
        
    elif format == 'pgvector':
        # PostgreSQL pgvector format: SQL INSERT statements
        sql_lines = ['-- PostgreSQL pgvector export', '-- Insert into embeddings table', '']
        
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            # Extract title for metadata
            title = chunk.split('\n')[0].replace('Article: ', '') if chunk.startswith('Article: ') else 'unknown'
            
            # Escape single quotes in text and title
            safe_chunk = chunk.replace("'", "''")
            safe_title = title.replace("'", "''")
            
            # Format embedding as PostgreSQL vector
            vector_str = '[' + ','.join(str(v) for v in embedding) + ']'
            
            # Build INSERT statement
            sql = f"INSERT INTO embeddings (chunk_id, chunk_text, embedding, source) VALUES ('{i}', E'{safe_chunk}', '{vector_str}'::vector, '{safe_title}');"
            sql_lines.append(sql)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(sql_lines))
        
        file_size_mb = __import__('os').path.getsize(output_path) / (1024 * 1024)
        
        print(f'✓ Exported {len(embeddings)} embeddings as PostgreSQL INSERT statements')
        print(f'  Output: {output_path}')
        print(f'  File size: {file_size_mb:.2f} MB')
        print(f'\nTo import into PostgreSQL:')
        print(f'  psql -U postgres -d your_database -f {output_path}')
        print(f'\nWorks with:')
        print(f'  - Local PostgreSQL + pgvector')
        print(f'  - Neon PostgreSQL')
        print(f'  - Supabase')
        print(f'  - AWS RDS with pgvector')
        
    elif format == 'pinecone':
        # Pinecone format: vectors with metadata
        export_data = {
            'vectors': [
                {
                    'id': f'chunk_{i}',
                    'values': embedding,
                    'metadata': {
                        'text': chunk,
                        'source': chunk.split('\n')[0].replace('Article: ', '') if chunk.startswith('Article: ') else 'unknown'
                    }
                }
                for i, (chunk, embedding) in enumerate(zip(chunks, embeddings))
            ]
        }
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)
        
        file_size_mb = __import__('os').path.getsize(output_path) / (1024 * 1024)
        
        print(f'✓ Exported {len(embeddings)} embeddings in Pinecone format')
        print(f'  Output: {output_path}')
        print(f'  File size: {file_size_mb:.2f} MB')
        print(f'\nTo import into Pinecone:')
        print(f'  1. Parse the JSON file')
        print(f'  2. Use Pinecone upsert API to insert vectors')
        
    else:
        raise ValueError(f"Unknown format '{format}'. Use 'generic', 'pgvector', or 'pinecone'")
    
    return {
        'count': len(embeddings),
        'dimension': embedding_dimension,
        'file_size_mb': file_size_mb,
        'format': format,
        'path': output_path
    }


# Example usage:
# Generic export (recommended for portability)
# stats = export_embeddings(format='generic')
# print(f"Exported {stats['count']} embeddings to {stats['path']}")

# PostgreSQL export
# stats = export_embeddings(format='pgvector')

# Pinecone export
# stats = export_embeddings(format='pinecone')

## Next Steps and Improvements

### Upgrade to Persistent Storage with PostgreSQL

**Current limitation**: Embeddings are lost when the notebook restarts, requiring 50+ minutes to regenerate.

**Solution**: Upgrade to the advanced version with PostgreSQL + pgvector:

1. **See** `wikipedia-rag-tutorial-advanced.ipynb` for the PostgreSQL version
2. **Benefits**:
   - Generate embeddings once, reuse across experiments
   - Store multiple embedding models for comparison
   - Run analyses in minutes instead of regenerating
   - Easy migration path to production databases

3. **Quick start**:
   ```bash
   # Start PostgreSQL
   docker run -d --name pgvector-rag \
     -e POSTGRES_PASSWORD=postgres \
     -e POSTGRES_DB=rag_db \
     -p 5432:5432 \
     -v pgvector_data:/var/lib/postgresql/data \
     pgvector/pgvector:pg16
   
   # Install PostgreSQL adapter
   pip install psycopg2-binary
   
   # Open the advanced notebook
   jupyter notebook wikipedia-rag-tutorial-advanced.ipynb
   ```

### Other RAG Improvements

1. **Hybrid Search**: Combine vector similarity with keyword search (BM25)
   - Better for specific terminology and exact matches
   - Combine results using reciprocal rank fusion

2. **Reranking**: Use a [reranking model](https://www.pinecone.io/learn/series/rag/rerankers/)
   - Cross-encoder models for better relevance
   - Re-score top 10-20 results from initial retrieval

3. **Query Expansion**: Generate multiple query variations
   - Use LLM to create related questions
   - Retrieve for each and merge results

4. **Better Chunking**:
   - Semantic chunking (split by meaning)
   - Overlapping chunks for better context
   - Parent-child chunks (retrieve child, return parent)

5. **Citation Support**:
   - Track which chunks were used
   - Provide Wikipedia URLs as sources
   - Show confidence scores

### Advanced RAG Patterns

- **Graph RAG**: Build knowledge graphs from Wikipedia links
- **Agentic RAG**: Let the LLM decide when to retrieve more information
- **Multi-hop RAG**: Follow reasoning chains across multiple documents
- **RAG Fusion**: Combine multiple retrieval strategies

### Learn More

- [HuggingFace RAG Guide](https://huggingface.co/blog/ngxson/make-your-own-rag)
- [Pinecone Learning Center](https://www.pinecone.io/learn/)
- Our documentation: See `POSTGRESQL_SETUP.md` for detailed setup instructions

## Dataset Statistics

View statistics about your loaded dataset:

In [ ]:
def print_dataset_stats():
    """Print statistics about the current dataset."""
    total_chars = sum(len(chunk) for chunk in dataset)
    avg_chunk_size = total_chars / len(dataset) if dataset else 0
    
    # Count unique articles
    articles = set()
    for chunk in dataset:
        if chunk.startswith('Article: '):
            title = chunk.split('\n')[0].replace('Article: ', '')
            articles.add(title)
    
    print('Dataset Statistics:')
    print(f'  Total chunks: {len(dataset):,}')
    print(f'  Unique articles: {len(articles):,}')
    print(f'  Total characters: {total_chars:,}')
    print(f'  Average chunk size: {avg_chunk_size:.0f} characters')
    print(f'  Estimated size: {sys.getsizeof(str(dataset)) / (1024*1024):.2f} MB')
    print(f'\n  Embeddings in database: {len(VECTOR_DB):,}')
    print(f'  Embedding dimension: {len(VECTOR_DB[0][1]) if VECTOR_DB else 0}')

print_dataset_stats()